In [ ]:
from os import path

import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
from astropy.io import ascii
from astropy.io import fits
from astropy.wcs import WCS
import reproject

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from scipy.spatial import cKDTree

from pyia import GaiaData
import gala.coordinates as gc
from dustmaps.sfd import SFDQuery

from sfd import reddening
from helpers import get_ext

In [ ]:
g = GaiaData('../data/all-data-3deg.fits')
c = g.get_skycoord(distance=False)

In [ ]:
DM = coord.Distance(31.5*u.kpc).distmod.value

In [ ]:
A_ps1 = reddening(c, 'PS1', 'grizy')
A_wise = reddening(c, 'WISE', ['W1', 'W2'])
A_galex = reddening(c, 'Galex', ['NUV', 'FUV'])
A_2mass = reddening(c, '2MASS', ['J', 'H', 'Ks'])
EBV = SFDQuery().query(c)

In [ ]:
A_gaia = get_ext(g.phot_g_mean_mag.value, 
                 g.phot_bp_mean_mag.value,
                 g.phot_rp_mean_mag.value,
                 EBV)
A_gaia = np.vstack(A_gaia).T

In [ ]:
phot = Table()

phot['G'] = g.phot_g_mean_mag.value - A_gaia[:, 0]
phot['BP'] = g.phot_bp_mean_mag.value - A_gaia[:, 1]
phot['RP'] = g.phot_rp_mean_mag.value - A_gaia[:, 2]

phot['g'] = g.ps1_g_mean_psf_mag - A_ps1[:, 0]
phot['r'] = g.ps1_r_mean_psf_mag.value - A_ps1[:, 1]
phot['i'] = g.ps1_i_mean_psf_mag.value - A_ps1[:, 2]
phot['z'] = g.ps1_z_mean_psf_mag.value - A_ps1[:, 3]
phot['y'] = g.ps1_y_mean_psf_mag.value - A_ps1[:, 4]

phot['W1'] = g.allwise_w1mpro - A_wise[:, 0]
phot['W2'] = g.allwise_w2mpro - A_wise[:, 1]

phot['NUV'] = g.galex_NUV.value - A_galex[:, 0]
phot['FUV'] = g.galex_FUV.value - A_galex[:, 1]

phot['J'] = g.data['2mass_j_m'] - A_2mass[:, 0]
phot['H'] = g.data['2mass_h_m'] - A_2mass[:, 1]
phot['Ks'] = g.data['2mass_ks_m'] - A_2mass[:, 2]

In [ ]:
# iso = Table.read('../data/FeH-1_all.fits')
iso = Table.read('../data/FeH-0.5_all.fits')

iso.rename_column('Gaia_G_DR2Rev', 'G')
iso.rename_column('Gaia_BP_DR2Rev', 'BP')
iso.rename_column('Gaia_RP_DR2Rev', 'RP')

for n in 'grizy':
    iso.rename_column('PS_'+n, n)

# iso.rename_column('WISE_W1', 'W1')
# iso.rename_column('WISE_W2', 'W2')

iso.rename_column('GALEX_FUV', 'FUV')
iso.rename_column('GALEX_NUV', 'NUV')

iso.rename_column('2MASS_J', 'J')
iso.rename_column('2MASS_H', 'H')
iso.rename_column('2MASS_Ks', 'Ks')

In [ ]:
col_mag = [('NUV', 'G', 'NUV'), ('NUV', 'i', 'NUV'), ('BP', 'RP', 'G'),
           ('BP', 'i', 'BP'), ('g', 'i', 'g'), ('r', 'y', 'r'),
           ('NUV', 'G', 'g', 'i'), ('NUV', 'i', 'BP', 'RP')]

xlims = [(-1.5, 4), (-1, 3), (-1., 0.75), 
         (-1, 0.75), (-1, 0.5), (-1, 0.5),
         (-2.5, 5), (-3, 4)]
ylims = [(22, 13), (22, 13), (20.5, 10), 
         (20.5, 10), (21.5, 13), (21.5, 13),
         (-1.5, 1), (-1.5, 1)]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
for k, row in enumerate(col_mag):
    ax = axes.flat[k]
    
    col = phot[row[0]] - phot[row[1]]
    i = iso[np.isclose(iso['isochrone_age_yr'], 5e7) & (iso['phase'] < 3)]
    
    if len(row) == 3:
        mag = phot[row[2]]
        isoy = i[row[2]] + DM
        ylabel = '${0[2]}$'.format(row)
        
    else:
        mag = phot[row[2]] - phot[row[3]]
        isoy = i[row[2]] - i[row[3]]
        ylabel = '${0[2]} - {0[3]}$'.format(row)
    
    ax.plot(col, mag, marker='.', linestyle='none', alpha=0.75)
    ax.plot(i[row[0]] - i[row[1]], isoy, marker='')
    
    ax.set_xlim(xlims[k])
    ax.set_ylim(ylims[k])
    
    ax.set_xlabel('${0[0]} - {0[1]}$'.format(row))
    ax.set_ylabel(ylabel)
    
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(phot['NUV']-phot['i'], 
        phot['BP']-phot['RP'],
        marker='.', linestyle='none')

i = iso[np.isclose(iso['isochrone_age_yr'], 5e7) & (iso['phase'] < 3)]
ax.plot(i['NUV']-i['i'], 
        i['BP']-i['RP'], 
        marker='')

nodes = np.array([[-1.2, -0.5],
                  [-1.2, -0.25],
                  [0, -0.1],
                  [1.5, 0.15],
                  [2, 0.4],
                  [2.5, 0.5],
                  [3.5, 0.5],
                  [2.5, 0.1],
                  [-1.2, -0.5]])
cc_path = mpl.patches.Path(nodes[:-1])
ax.plot(nodes[:,0], nodes[:,1])

ax.set_xlim(-3, 4)
ax.set_ylim(-1, 1)

ax.set_xlabel('$NUV - i$')
ax.set_ylabel('$G_{BP} - G_{RP}$')

In [ ]:
X = np.vstack((phot['NUV']-phot['i'], 
               phot['BP']-phot['RP'])).T
cc_mask = cc_path.contains_points(X)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
ax.plot(g.ra, g.dec, marker=',', color='#aaaaaa', alpha=0.5, linestyle='none')
ax.plot(g.ra[cc_mask], g.dec[cc_mask], 
        marker='o', color='tab:blue', 
        alpha=0.5, linestyle='none')

ax = axes[1]
ax.plot(g.pmra, g.pmdec, marker=',', color='#aaaaaa', alpha=0.75, linestyle='none')
ax.plot(g.pmra[cc_mask], g.pmdec[cc_mask], 
        marker='o', color='tab:blue', 
        alpha=0.5, linestyle='none')
ax.scatter(-0.6, 0.5, color='tab:red', zorder=100)
ax.scatter(-0.6, 0.5+0.5, color='tab:red', zorder=100)
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

fig.tight_layout()

In [ ]:
pm_mask = np.sqrt((g.pmra - -0.6*u.mas/u.yr)**2 + (g.pmdec - 0.5*u.mas/u.yr)**2) < 0.5*u.mas/u.yr

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot((phot['g']-phot['i'])[pm_mask], 
        (phot['r']-phot['z'])[pm_mask],
        marker='.', linestyle='none')
ax.plot((phot['g']-phot['i'])[pm_mask & cc_mask], 
        (phot['r']-phot['z'])[pm_mask & cc_mask],
        marker='.', linestyle='none')

i = iso[np.isclose(iso['isochrone_age_yr'], 5e7) & (iso['phase'] < 3)]
ax.plot(i['g']-i['i'], 
        i['r']-i['z'], 
        marker='')

# nodes = np.array([[-2.5, -0.75],
#                   [-2.5, -0.5],
#                   [0, -0.1],
#                   [2, 0.4],
#                   [2.5, 0.5],
#                   [3.5, 0.5],
#                   [2.5, 0.1],
#                   [-2.5, -0.75]])
# cc_path = mpl.patches.Path(nodes[:-1])
# ax.plot(nodes[:,0], nodes[:,1])

ax.set_xlim(-1, 0)
ax.set_ylim(-0.7, 0.5)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot((phot['NUV']-phot['i'])[pm_mask], 
        (phot['BP']-phot['RP'])[pm_mask],
        marker='.', linestyle='none')

i = iso[np.isclose(iso['isochrone_age_yr'], 5e7) & (iso['phase'] < 3)]
ax.plot(i['NUV']-i['i'], 
        i['BP']-i['RP'], 
        marker='')

nodes = np.array([[-2.5, -0.75],
                  [-2.5, -0.5],
                  [0, -0.1],
                  [2, 0.4],
                  [2.5, 0.5],
                  [3.5, 0.5],
                  [2.5, 0.1],
                  [-2.5, -0.75]])
cc_path = mpl.patches.Path(nodes[:-1])
ax.plot(nodes[:,0], nodes[:,1])

ax.set_xlim(-3, 4)
ax.set_ylim(-1, 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4.8))

ax = axes[0]
ax.plot(g.ra, g.dec, marker=',', color='#aaaaaa', alpha=1, linestyle='none')
ax.plot(g.ra[cc_mask&pm_mask], g.dec[cc_mask&pm_mask], 
        marker='o', color='tab:blue', 
        alpha=0.5, linestyle='none')
ax.set_xlim(180, 178)
ax.set_ylim(-30.5, -28.5)

ax = axes[1]
ax.plot(g.ra, g.dec, marker=',', color='#aaaaaa', alpha=1, linestyle='none')
ax.plot(g.ra[cc_mask&pm_mask], g.dec[cc_mask&pm_mask], 
        marker='o', color='tab:blue', 
        alpha=0.5, linestyle='none')
ax.set_xlim(181, 179)
ax.set_ylim(-29, -27)

fig.tight_layout()